In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
MFCCs = np.load("../preprocessing/MFCCs/MFCC.npy")

In [3]:
print(np.shape(MFCCs))

MFCCs = (MFCCs - np.mean(MFCCs))/np.std(MFCCs)

(400, 20, 6000)


In [4]:
info_songs = pd.read_csv("../Info/info.csv")

In [5]:
label_songs = pd.read_csv("../preprocessing/labels.csv")

In [6]:
select_label = "genre"

if select_label == "genre":
    labels = label_songs[select_label].map({"classical":0, "electronic":1, "pop":2, "rock":3})
labels = labels.to_numpy()

In [7]:
length_songs = pd.DataFrame(info_songs["Track ID"])
length_songs["Song length"] = info_songs["Duration"]*10
length_songs["Song length"] = length_songs["Song length"].astype(int)
length_songs

,Track ID,Song length
0,1,600
1,10,600
2,100,600
3,11,600
4,12,600
...,...,...
395,395,600
396,396,600
397,397,600
398,398,600


In [8]:
n_data = len(length_songs)
idx = np.random.permutation(n_data)
length_songs = length_songs.reindex(idx)["Song length"].to_numpy()
MFCCs = MFCCs[idx, :, :]
labels = labels[idx]

In [9]:
import tensorflow as tf

In [11]:
fraction_validation = 0.25
n_test = int(fraction_validation*len(labels))
n_train = len(labels) - n_test

train_features, train_labels, train_length = MFCCs[:n_train], labels[:n_train], length_songs[:n_train]
validation_features, validation_labels, validation_length = MFCCs[n_train:], labels[n_train:], length_songs[n_train:]

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels, train_length))
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_features, validation_labels, validation_length))

In [35]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import Input, layers
from tensorflow.keras import backend as K

sample_size = (20, 50, 1)

drop_out_rate = 0.2

# very simple keras Sequential model
input_tensor = Input(sample_size)
x = layers.Conv2D(16, (1, 10), padding="valid", activation="relu", strides=1)(input_tensor)
x = layers.Conv2D(32, (1, 5), padding="valid", activation="relu", strides=1)(x)
x = layers.Dropout(drop_out_rate)(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(drop_out_rate)(x)
output_tensor = layers.Dense(4, activation="softmax")(x)

model_1 = tf.keras.Model(input_tensor, output_tensor)

In [36]:
model_1.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 20, 50, 1)]       0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 20, 41, 16)        176       
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 20, 37, 32)        2592      
_________________________________________________________________
dropout_14 (Dropout)         (None, 20, 37, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 10, 18, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 5760)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               1474

In [65]:
input_tensor = Input(sample_size)
x = layers.Conv2D(16, (1, 10), padding="valid", activation="relu", strides=1)(input_tensor)
x = layers.Conv2D(32, (1, 15), padding="valid", activation="relu", strides=1)(x)
x = layers.Dropout(drop_out_rate)(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Conv2D(32, (2,2), padding="valid", activation="relu", strides=1)(x)
x = layers.Flatten()(x)
x = layers.Dropout(2*drop_out_rate)(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(drop_out_rate)(x)
output_tensor = layers.Dense(4, activation="softmax")(x)

model_2 = tf.keras.Model(input_tensor, output_tensor)
model_2.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        [(None, 20, 50, 1)]       0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 20, 41, 16)        176       
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 20, 27, 32)        7712      
_________________________________________________________________
dropout_31 (Dropout)         (None, 20, 27, 32)        0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 10, 13, 32)        0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 9, 12, 32)         4128      
_________________________________________________________________
flatten_22 (Flatten)         (None, 3456)              0  

In [80]:
input_tensor = Input(sample_size)
x = layers.Conv2D(16, (3, 3), padding="valid", activation="relu", strides=1)(input_tensor)
x = layers.MaxPooling2D((2,2))(x)

x = layers.Conv2D(32, (3, 3), padding="valid", activation="relu", strides=1)(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)

x = layers.Conv2D(32, (1,2), padding="valid", activation="relu", strides=1)(x)
x = layers.MaxPooling2D(pool_size=(1,2))(x)

x = layers.Conv2D(16, (2,2), padding="valid", activation="relu", strides=1)(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)

x = layers.Flatten()(x)

x = layers.Dropout(2*drop_out_rate)(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(drop_out_rate)(x)
output_tensor = layers.Dense(4, activation="softmax")(x)

model_3 = tf.keras.Model(input_tensor, output_tensor)
model_3.summary()

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_41 (InputLayer)        [(None, 20, 50, 1)]       0         
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 18, 48, 16)        160       
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 9, 24, 16)         0         
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 7, 22, 32)         4640      
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 3, 11, 32)         0         
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 3, 10, 32)         2080      
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 3, 5, 32)          0  

In [81]:
#model = model_1
model = model_2
model = model_3

model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [82]:
def slice_accordingly(input_tensor, labels, size=sample_size):

    input_shape = (tf.shape(input_tensor).numpy())
    
    input_shape[1], input_shape[2] = size[0], size[1]
    size = input_shape
    
    sliced_tensor = tf.image.random_crop(input_tensor, size)
    
    nan_values, idx = tf.unique(tf.gather(tf.where(tf.math.is_nan(sliced_tensor)), 0, axis=1))
    nan_values = nan_values.numpy()
  
    msk = np.zeros((input_shape[0]), dtype=np.bool)
    msk[nan_values] = True
    msk = ~msk

    sliced_tensor = tf.boolean_mask(sliced_tensor, msk , axis=0)
    labels = tf.boolean_mask(labels , msk, axis=0)
    
    return sliced_tensor, labels

In [83]:
n_epochs = 100
batch_size = 50

for epoch in range(n_epochs):
    print("Epoch", epoch)
    train_ds = train_dataset.shuffle(n_train).batch(batch_size)
    val_ds = validation_dataset.shuffle(n_test).batch(n_test)
    
    x_val, y_val, length_val = next(iter(val_ds))
    x_val, y_val = slice_accordingly(x_val, y_val)
    
    for features, labels, lengths in train_ds:
        features, labels = slice_accordingly(features, labels)
        
        history = model.fit(features, labels,
                           validation_data=(x_val, y_val),
                           verbose=1) 
        predictions = model.predict(features)

Epoch 0
2/2 [==============================] - 0s 107ms/step - loss: 1.9950 - accuracy: 0.3000 - val_loss: 1.5014 - val_accuracy: 0.3030
Epoch 1
2/2 [==============================] - 0s 104ms/step - loss: 1.5301 - accuracy: 0.2800 - val_loss: 1.4179 - val_accuracy: 0.2300
Epoch 2
2/2 [==============================] - 0s 105ms/step - loss: 1.3800 - accuracy: 0.3000 - val_loss: 1.3837 - val_accuracy: 0.2600
Epoch 3
2/2 [==============================] - 0s 91ms/step - loss: 1.3368 - accuracy: 0.3600 - val_loss: 1.3593 - val_accuracy: 0.3814
Epoch 4
2/2 [==============================] - 0s 88ms/step - loss: 1.3737 - accuracy: 0.3200 - val_loss: 1.3416 - val_accuracy: 0.3469
Epoch 5
2/2 [==============================] - 0s 104ms/step - loss: 1.2992 - accuracy: 0.3600 - val_loss: 1.3353 - val_accuracy: 0.3608
Epoch 6
2/2 [==============================] - 0s 92ms/step - loss: 1.3705 - accuracy: 0.2600 - val_loss: 1.3381 - val_accuracy: 0.3673
Epoch 7
2/2 [==============================]